In [ ]:
# this is the method Gagandip was using

streak_cols = ['home_win','away_win','draw']

for col in streak_cols:
    condition = fulldf[col].eq(0)
    fulldf['reset'] = condition.groupby([fulldf['home_team'],fulldf['season']]).cumsum()
    fulldf[col + '_streak'] = fulldf[col].mask(condition, 0).groupby([fulldf['home_team'],fulldf['reset']]).cumsum()

for col in streak_cols:
    condition = fulldf[col].eq(0)
    fulldf['reset'] = condition.groupby([fulldf['away_team'],fulldf['season']]).cumsum()
    fulldf[col + '_streak'] = fulldf[col].mask(condition, 0).groupby([fulldf['away_team'],fulldf['reset']]).cumsum()

# test

fulldf[(fulldf['season']=='2018') & (fulldf['home_team']=='brighton-amp-hov')]

## A third attempt

See https://stackoverflow.com/questions/42397647/pythonic-way-to-calculate-streaks-in-pandas-dataframe

In [ ]:
# test with one season / team
bh = fulldf[(fulldf['season']=='2018') & (fulldf['home_team']=='brighton-amp-hov')]
s = bh['home_win'].groupby((bh['home_win'] != bh['home_win'].shift()).cumsum()).cumsum()
bh.assign(streak=s.where(s>0, 0))

In [ ]:
def streaks(df,cols):
    kwargs = {}
    for col in cols:
        s = df[col].groupby((df[col]!=df[col].shift()).cumsum()).cumsum()
        kwargs['cum_' + col] = s.where(s>0, 0)
    return df.assign(**kwargs)


In [ ]:
# check it works OK
streaks(fulldf[(fulldf['season']==str(2018)) & (fulldf['home_team']=='brighton-amp-hov')], ['home_win','away_win','draw'])

In [ ]:
l = []
for season in range(1990,2022):
    for team in teams:
        l.append(streaks(fulldf[(fulldf['season']==str(season)) & (fulldf['home_team']==team)], ['home_win','away_win','draw']))
home_streaks_df = pd.concat(l)

In [ ]:
# again, check it's working
home_streaks_df[(home_streaks_df['season']=='2018') & (home_streaks_df['home_team']=='brighton-amp-hov')]